In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
def rec_walk(path, del_ren):
    for entry in os.listdir(path):
        fullpath = os.path.join(path, entry)
        if str(entry).startswith("df_"):
            if del_ren == "del" and ((".old" not in fullpath) or (".fil" not in fullpath)):
                print(f"delete {fullpath}")
                os.remove(fullpath)
            elif del_ren == "ren":
                if ".old2" in fullpath:
                    print(f"renamed {fullpath} removing .old2")
                    os.rename(fullpath, fullpath[:-(len(".old2"))])
                elif ".old" not in fullpath:
                    print(f"renamed {fullpath} to .fil")
                    os.rename(fullpath, fullpath+".fil")
                # print(f"renamed {fullpath} to + .old")
                # os.rename(fullpath, fullpath+".old")

                # if ".old.old" in fullpath:
                #     print("removed .old.old")
                #     os.rename(fullpath, fullpath[:-(len(".old"))])
            else:
                print(f"skipped {fullpath}")
        elif(os.path.isdir(fullpath)):
            rec_walk(fullpath, del_ren)

# rec_walk('./output/manhattan', 'ren')
# rec_walk('./output/all_equipped', 'del')

In [ ]:
def read_data(root):
    if os.path.isfile(f'{root}/df_green.txt.gz') and os.path.isfile(f'{root}/df_sem_time.txt.gz') and os.path.isfile(f'{root}/df_trip_time.txt.gz'):
        df_green = pd.read_csv(f'{root}/df_green.txt.gz')
        df_sem_time = pd.read_csv(f'{root}/df_sem_time.txt.gz')
        df_trip_time = pd.read_csv(f'{root}/df_trip_time.txt.gz')
        print(f'LOADED: {root}')
    else:
        df_green = pd.DataFrame()
        df_sem_time = pd.DataFrame()
        df_trip_time = pd.DataFrame()
        max_plans = len(list(filter(lambda d: os.path.isdir(os.path.abspath(f'{root}/{d}')), os.listdir(os.path.abspath(root))))) #5 if "exp2" in root else 20
        for i in range(0,max_plans):
            df_green_ = pd.read_csv(f'{root}/{i}/greenwave.txt.gz', sep='\t')
            df_green_.dropna(inplace=True)
            df_green_['run'] = i
            df_green = pd.concat([df_green, df_green_])
            df_sem_time_ = pd.read_csv(f'{root}/{i}/semaphore_time.txt.gz', sep='\t')
            df_sem_time_.dropna(inplace=True)
            df_sem_time_['run'] = i
            df_sem_time = pd.concat([df_sem_time, df_sem_time_])
            df_trip_time_ = pd.read_csv(f'{root}/{i}/ITERS/it.0/0.tripdurations.txt', sep='\t', on_bad_lines='skip')
            df_trip_time_.dropna(inplace=True)
            df_trip_time_['run'] = i
            df_trip_time = pd.concat([df_trip_time, df_trip_time_])
        df_green.to_csv(f'{root}/df_green.txt.gz', index=False)
        df_sem_time.to_csv(f'{root}/df_sem_time.txt.gz', index=False)
        df_trip_time.to_csv(f'{root}/df_trip_time.txt.gz', index=False)
        print(f'CREATED: {root}')
        

    return df_green, df_sem_time, df_trip_time

In [ ]:
def get_dataframes(x_list, root):
    if os.path.isfile(f'{root}/df_green.txt.gz') and os.path.isfile(f'{root}/df_sem_time.txt.gz') and os.path.isfile(f'{root}/df_trip_time.txt.gz'):
        df_green = pd.read_csv(f'{root}/df_green.txt.gz')
        df_sem_time = pd.read_csv(f'{root}/df_sem_time.txt.gz')
        df_trip_time = pd.read_csv(f'{root}/df_trip_time.txt.gz')
        print(f'LOADED: {root}')
    else:
        df_green = pd.DataFrame()
        df_sem_time = pd.DataFrame()
        df_trip_time = pd.DataFrame()
        for x_label in x_list:
            labels = x_label[0]
            to_load = x_label[1] # exp0traffic0000/000smartAgents
            dfs_ = read_data(to_load)
            for df_ in dfs_:
                for name, value in labels.items(): # labels == { %equip: x, mode: 'basic' }
                    df_[name] = value

            df_green = pd.concat([df_green, dfs_[0]])
            df_sem_time = pd.concat([df_sem_time, dfs_[1]])
            df_trip_time = pd.concat([df_trip_time, dfs_[2]])

        df_green.to_csv(f'{root}/df_green.txt.gz', index=False)
        df_sem_time.to_csv(f'{root}/df_sem_time.txt.gz', index=False)
        df_trip_time.to_csv(f'{root}/df_trip_time.txt.gz', index=False)
        print(f'CREATED: {root}')
    
    return df_green, df_sem_time, df_trip_time

In [ ]:
palette ={"fixed": "tab:orange", "basic": "tab:blue", "communication": "tab:green", "proportional": "tab:red", "communication NO Propagation": "tab:pink", "proportional NO Propagation": "tab:purple"}
labels_map ={"fixed": "FTC", "basic": "Basic", "communication": "Coordinated", "proportional": "Coordinated−Prop", "communication NO Propagation": "Coordinated−NO Propagation", "proportional NO Propagation": "Coordinated−Prop−NO Propagation"}
def map_labels(lab):
    return [labels_map[x] for x in lab]

In [ ]:
def draw_plots_box(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, x, exp_str):
    df_trip_time_manhattan = df_trip_time_manhattan[df_trip_time_manhattan["Attributes"].str.contains('travelTime')]
    # df_trip_time_masa = df_trip_time_masa[df_trip_time_masa["Attributes"].str.contains('travelTime')]
    df_trip_time_manhattan['DTime'] = df_trip_time_manhattan["Attributes"].apply(lambda x:
        float(dict(
            map(lambda y: (y.split('=')[0], y.split('=')[1]) ,
                x.split(';'))
            )["travelTime"]
        )
    )
    # df_trip_time_masa['DTime'] = df_trip_time_masa["Attributes"].apply(lambda x:float(dict(map(lambda y:(y.split('=')[0], y.split('=')[1]) ,x.split(';')))["travelTime"]))   
    df_trip_time_manhattan["DDuration"] = df_trip_time_manhattan['Duration'] / df_trip_time_manhattan['DTime']
    # df_trip_time_masa["DDuration"] = df_trip_time_masa['Duration'] / df_trip_time_masa['DTime']

    # df_green_masa["Ratio100"] = df_green_masa["Ratio"] * 100
    df_green_manhattan["Ratio100"] = df_green_manhattan["Ratio"] * 100

    if exp_str == 'high traffic':
        exp_str = 'heavy traffic'

    fig = plt.figure(figsize=(20, 18), dpi = 400)
    #fig.suptitle(exp_str)
    axs = fig.subplots(3, 2)
    #axs = figs[0].subplots(2, 1)
    #figs[0].suptitle("Time To Cross")
    axs[0,0].set_xticklabels(axs[0,0].get_xticks())
    sns.boxplot(data=df_sem_time_manhattan, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,0], showfliers = False, linewidth=2)
    handles, labels = axs[0,0].get_legend_handles_labels()
    sns.stripplot(data=df_sem_time_manhattan, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,0], dodge=True, jitter=False, marker="d", zorder=0, size=3)
    axs[0,0].set(yscale = 'log')
    axs[0,0].legend().remove()
    axs[0,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[0,0].set_ylabel("Time (s)")
    axs[0,0].set_xlabel("n° vehicles")
    axs[0,0].set_title(f"Time to Cross − Manhattan − {exp_str} (a)")
    #ax = axs[0].subplot(1,2,2)
    #axs[1].xticks(rotation = 45)
    axs[0,1].set_xticklabels(axs[0,1].get_xticks())
    # sns.boxplot(data=df_sem_time_masa, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,1], showfliers = False, linewidth=2)
    handles, labels = axs[0,1].get_legend_handles_labels()
    # sns.stripplot(data=df_sem_time_masa, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,1], dodge=True, jitter=False, marker="d", zorder=0, size=3)
    axs[0,1].set(yscale = 'log')
    axs[0,1].legend().remove()
    axs[0,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #axs[1].figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.06), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[0,1].set_ylabel("Time (s)")
    axs[0,1].set_xlabel("n° vehicles")
    axs[0,1].set_title(f"Time to Cross − MASA − {exp_str} (b)")
    #axs[0].suptitle(exp_str + " - Time to cross traffic lights")
    #plt.figure(figsize=(8, 6), dpi = 1200)
    #plt.subplot(1,2,1)
    #plt.xticks(rotation = 45)
    #axs = figs[1].subplots(2, 1)
    #figs[1].suptitle("Green Traffic Lights Ratio")
    axs[1,0].set_xticklabels(axs[1,0].get_xticks())
    sns.boxplot(data=df_green_manhattan.query("Mode != 'fixed'"), x=x, y='Ratio100', hue='Mode', palette=palette, ax=axs[1,0], showfliers = False)
    handles, labels = axs[1,0].get_legend_handles_labels()
    axs[1,0].legend().remove()
    axs[1,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[1,0].set_ylabel("% of green traffic lights")
    axs[1,0].set_xlabel("n° vehicles")
    axs[1,0].set_title(f"Green Traffic Lights Percentage − Manhattan − {exp_str} (c)")
    #ax = plt.subplot(1,2,2)
    #plt.xticks(rotation = 45)
    axs[1,1].set_xticklabels(axs[1,1].get_xticks())
    # sns.boxplot(data=df_green_masa.query("Mode != 'fixed'"), x=x, y='Ratio100', hue='Mode', palette=palette, ax=axs[1,1], showfliers = False)
    #handles, labels = ax.get_legend_handles_labels()
    handles, labels = axs[1,1].get_legend_handles_labels()
    axs[1,1].legend().remove()
    axs[1,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #plt.figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.06), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[1,1].set_ylabel("% of green traffic lights")
    axs[1,1].set_xlabel("n° vehicles")
    axs[1,1].set_title(f"Green Traffic Lights Percentage − MASA − {exp_str} (d)")
    #plt.suptitle(exp_str + " - Encurred green traffic lights")
    #plt.figure(figsize=(8, 6), dpi = 1200)
    #plt.subplot(1,2,1)
    #plt.xticks(rotation = 45)
    #axs = figs[2].subplots(2, 1)
    #axs[0].set_xticklabels(axs[0].get_xticks(), rotation = 45)
    #figs[2].suptitle("Trip Time Slowdown")
    sns.boxplot(data=df_trip_time_manhattan, x=x, y='DDuration', hue='Mode', palette=palette, ax=axs[2,0], showfliers = False)
    axs[2,0].set(yscale = 'log')
    handles, labels = axs[2,0].get_legend_handles_labels()
    axs[2,0].legend().remove()
    axs[2,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[2,0].set_ylabel("Slowdown")
    axs[2,0].set_xlabel("n° vehicles")
    axs[2,0].set_title(f"Trip Time Slowdown − Manahattan − {exp_str} (e)")
    # qs = df_trip_time_manhattan.query("Mode == 'communication'")["DDuration"].quantile([0.25, 0.75])
    # iqr = qs[0.75] - qs[0.25]
    # axs[0].axis(ymin=qs[0.25] - 2*iqr, ymax=qs[0.75] + 1.5*iqr)
    #ax = plt.subplot(1,2,2)
    #plt.xticks(rotation = 45)
    axs[2,1].set_xticklabels(axs[2,1].get_xticks())
    # sns.boxplot(data=df_trip_time_masa, x=x, y='DDuration', hue='Mode', palette=palette, ax=axs[2,1], showfliers = False)
    axs[2,1].set(yscale = 'log')
    #handles, labels = ax.get_legend_handles_labels()
    handles, labels = axs[2,1].get_legend_handles_labels()
    axs[2,1].legend().remove()
    axs[2,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #plt.figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.1), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[2,1].set_ylabel("Slowdown")
    axs[2,1].set_xlabel("n° vehicles")
    axs[2,1].set_title(f"Trip Time Slowdown − MASA − {exp_str} (f)")
    fig.tight_layout(h_pad=2.5)


In [ ]:
def load(traffic):
    # ROOT = f'Manhattan/output/coexist_{traffic}'
    traffic = 1000 if traffic == 'low' else 2500 if traffic == 'medium' else 4000
    # ROOT = f'output/manhattan/exp1'
    ROOT = f'output/all_equipped'
    modes = [ 'basic','fixed', 'communication', 'proportional']
    # modes = ['fixed']
    n_vehicles = range(1000, 5001, 500)
    # n_vehicles = range(100, 1001, 100)
    df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = get_dataframes([({'Traffic': x, 'Mode': mode}, f'{ROOT}/{mode if mode != "fixed" else "ftc"}/{x}agents') for x in n_vehicles for mode in modes], ROOT)

    # ROOT = f'MASA/output/coexist_{traffic}'
    # df_green_masa, df_sem_time_masa, df_trip_time_masa = get_dataframes([({'% Equipped': x, 'Mode': mode}, f'{ROOT}/{mode}/{x}') for x in range(20,81,20) for mode in ['basic', 'fixed', 'communication', 'proportional']], ROOT)

    return (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan)#, (df_green_masa, df_sem_time_masa, df_trip_time_masa)

def isEquipped(attrs, sep):
    attrs_list = attrs.split(sep)
    for att in attrs_list:
        k,v = att.split('=')
        if k == 'drivelogic':
            return 'Bid' in v

def flagEquipped(df_green, df_sem_time, df_trip_time):
    df_trip_time["Equipped"] = df_trip_time["Attributes"].map(lambda x:isEquipped(x, ';'))

    df_trip_time = df_trip_time.astype({'Traffic': 'int', 'Equipped': 'int', 'Person': 'int'})
    df_sem_time = df_sem_time.astype({'Traffic': 'int', 'Agent': 'int'})
    df_green = df_green.astype({'Traffic': 'int', 'Agent': 'int'})

    df_sem_time = df_sem_time.merge(df_trip_time, how='left', left_on=['run', 'Agent', 'Traffic', 'Mode'], right_on=['run', 'Person', 'Traffic', 'Mode'])[['Link', 'Time', 'Agent', 'Start', 'AgentAttrs', 'run', 'Traffic',
       'Mode', 'Equipped']]
    df_green = df_green.merge(df_trip_time, how='left', left_on=['run', 'Agent', 'Traffic', 'Mode'], right_on=['run', 'Person', 'Traffic', 'Mode'])[['Agent', 'Ratio', 'run', 'Equipped', 'Mode', 'Traffic']]
    return df_green, df_sem_time, df_trip_time

def equipped(manhattan, masa=None):
    # df_green_masa, df_sem_time_masa, df_trip_time_masa = flagEquipped(*masa)
    df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = flagEquipped(*manhattan)

    return (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan)#, (df_green_masa, df_sem_time_masa, df_trip_time_masa)

In [ ]:

def traffic(t):
    # manhattan, masa = load(t)
    manhattan = load(t)
    # (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan), (df_green_masa, df_sem_time_masa, df_trip_time_masa) = equipped(manhattan, masa)
    (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan) = equipped(manhattan)
    if t == 'high':
        t = 'heavy'
    #draw_plots(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, df_green_masa, df_sem_time_masa, df_trip_time_masa, "% Equipped", f"Coexist - High {t}")
    draw_plots_box(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, "Traffic", f"All Equipped − {t} traffic")
    #draw_plots_box_detail(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, df_green_masa, df_sem_time_masa, df_trip_time_masa, "% Equipped", f"Coexist - High {t}")


In [ ]:
traffic('high')

In [ ]:
def draw_plots_box(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, x, exp_str):
    df_trip_time_manhattan = df_trip_time_manhattan[df_trip_time_manhattan["Attributes"].str.contains('travelTime')]
    # df_trip_time_masa = df_trip_time_masa[df_trip_time_masa["Attributes"].str.contains('travelTime')]
    df_trip_time_manhattan['DTime'] = df_trip_time_manhattan["Attributes"].apply(lambda x:float(dict(map(lambda y:(y.split('=')[0], y.split('=')[1]) ,x.split(';')))["travelTime"]))
    # df_trip_time_masa['DTime'] = df_trip_time_masa["Attributes"].apply(lambda x:float(dict(map(lambda y:(y.split('=')[0], y.split('=')[1]) ,x.split(';')))["travelTime"]))   
    df_trip_time_manhattan["DDuration"] = df_trip_time_manhattan['Duration'] / df_trip_time_manhattan['DTime']
    # df_trip_time_masa["DDuration"] = df_trip_time_masa['Duration'] / df_trip_time_masa['DTime']

    # df_green_masa["Ratio100"] = df_green_masa["Ratio"] * 100
    df_green_manhattan["Ratio100"] = df_green_manhattan["Ratio"] * 100

    if exp_str == 'high traffic':
        exp_str = 'heavy traffic'

    fig = plt.figure(figsize=(20, 18), dpi = 400)
    #fig.suptitle(exp_str)
    axs = fig.subplots(3, 2)
    #axs = figs[0].subplots(2, 1)
    #figs[0].suptitle("Time To Cross")
    axs[0,0].set_xticklabels(axs[0,0].get_xticks())
    sns.boxplot(data=df_sem_time_manhattan, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,0], showfliers = False, linewidth=2)
    handles, labels = axs[0,0].get_legend_handles_labels()
    sns.stripplot(data=df_sem_time_manhattan, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,0], dodge=True, jitter=False, marker="d", zorder=0, size=3)
    axs[0,0].set(yscale = 'log')
    axs[0,0].legend().remove()
    axs[0,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[0,0].set_ylabel("Time (s)")
    axs[0,0].set_xlabel("% Connected")
    axs[0,0].set_title(f"Time to Cross − Manhattan − {exp_str} (a)")
    #ax = axs[0].subplot(1,2,2)
    #axs[1].xticks(rotation = 45)
    axs[0,1].set_xticklabels(axs[0,1].get_xticks())
    # sns.boxplot(data=df_sem_time_masa, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,1], showfliers = False, linewidth=2)
    handles, labels = axs[0,1].get_legend_handles_labels()
    # sns.stripplot(data=df_sem_time_masa, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,1], dodge=True, jitter=False, marker="d", zorder=0, size=3)
    axs[0,1].set(yscale = 'log')
    axs[0,1].legend().remove()
    axs[0,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #axs[1].figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.06), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[0,1].set_ylabel("Time (s)")
    axs[0,1].set_xlabel("% Connected")
    axs[0,1].set_title(f"Time to Cross − MASA − {exp_str} (b)")
    #axs[0].suptitle(exp_str + " - Time to cross traffic lights")
    #plt.figure(figsize=(8, 6), dpi = 1200)
    #plt.subplot(1,2,1)
    #plt.xticks(rotation = 45)
    #axs = figs[1].subplots(2, 1)
    #figs[1].suptitle("Green Traffic Lights Ratio")
    axs[1,0].set_xticklabels(axs[1,0].get_xticks())
    sns.boxplot(data=df_green_manhattan.query("Mode != 'fixed'"), x=x, y='Ratio100', hue='Mode', palette=palette, ax=axs[1,0], showfliers = False)
    handles, labels = axs[1,0].get_legend_handles_labels()
    axs[1,0].legend().remove()
    axs[1,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[1,0].set_ylabel("% of green traffic lights")
    axs[1,0].set_xlabel("% Connected")
    axs[1,0].set_title(f"Green Traffic Lights Percentage − Manhattan − {exp_str} (c)")
    #ax = plt.subplot(1,2,2)
    #plt.xticks(rotation = 45)
    axs[1,1].set_xticklabels(axs[1,1].get_xticks())
    # sns.boxplot(data=df_green_masa.query("Mode != 'fixed'"), x=x, y='Ratio100', hue='Mode', palette=palette, ax=axs[1,1], showfliers = False)
    #handles, labels = ax.get_legend_handles_labels()
    handles, labels = axs[1,1].get_legend_handles_labels()
    axs[1,1].legend().remove()
    axs[1,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #plt.figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.06), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[1,1].set_ylabel("% of green traffic lights")
    axs[1,1].set_xlabel("% Connected")
    axs[1,1].set_title(f"Green Traffic Lights Percentage − MASA − {exp_str} (d)")
    #plt.suptitle(exp_str + " - Encurred green traffic lights")
    #plt.figure(figsize=(8, 6), dpi = 1200)
    #plt.subplot(1,2,1)
    #plt.xticks(rotation = 45)
    #axs = figs[2].subplots(2, 1)
    #axs[0].set_xticklabels(axs[0].get_xticks(), rotation = 45)
    #figs[2].suptitle("Trip Time Slowdown")
    sns.boxplot(data=df_trip_time_manhattan, x=x, y='DDuration', hue='Mode', palette=palette, ax=axs[2,0], showfliers = False)
    axs[2,0].set(yscale = 'log')
    handles, labels = axs[2,0].get_legend_handles_labels()
    axs[2,0].legend().remove()
    axs[2,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[2,0].set_ylabel("Slowdown")
    axs[2,0].set_xlabel("% Connected")
    axs[2,0].set_title(f"Trip Time Slowdown − Manahattan − {exp_str} (e)")
    # qs = df_trip_time_manhattan.query("Mode == 'communication'")["DDuration"].quantile([0.25, 0.75])
    # iqr = qs[0.75] - qs[0.25]
    # axs[0].axis(ymin=qs[0.25] - 2*iqr, ymax=qs[0.75] + 1.5*iqr)
    #ax = plt.subplot(1,2,2)
    #plt.xticks(rotation = 45)
    axs[2,1].set_xticklabels(axs[2,1].get_xticks())
    # sns.boxplot(data=df_trip_time_masa, x=x, y='DDuration', hue='Mode', palette=palette, ax=axs[2,1], showfliers = False)
    axs[2,1].set(yscale = 'log')
    #handles, labels = ax.get_legend_handles_labels()
    handles, labels = axs[2,1].get_legend_handles_labels()
    axs[2,1].legend().remove()
    axs[2,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #plt.figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.1), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[2,1].set_ylabel("Slowdown")
    axs[2,1].set_xlabel("% Connected")
    axs[2,1].set_title(f"Trip Time Slowdown − MASA − {exp_str} (f)")
    fig.tight_layout(h_pad=2.5)


In [ ]:
def load(traffic):
    # ROOT = f'Manhattan/output/coexist_{traffic}'
    # traffic = 1000 if traffic == 'low' else 2500 if traffic == 'medium' else 4000
    # ROOT = f'output/manhattan/exp2traffic{traffic}'
    ROOT = f'output/coexist_{traffic}'
    modes = ['basic', 'fixed', 'communication', 'proportional']
    # modes = ['fixed']
    smart_perc = range(20, 81, 20)
    # df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = get_dataframes([({'% Equipped': x, 'Mode': mode}, f'{ROOT}/{mode if mode != "fixed" else "ftc"}/{int(x*traffic/100)}smart_agents') for x in smart_perc for mode in modes], ROOT)
    df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = get_dataframes([({'% Equipped': x, 'Mode': mode}, f'{ROOT}/{mode if mode != "fixed" else "ftc"}/{x}agents') for x in smart_perc for mode in modes], ROOT)

    # ROOT = f'MASA/output/coexist_{traffic}'
    # df_green_masa, df_sem_time_masa, df_trip_time_masa = get_dataframes([({'% Equipped': x, 'Mode': mode}, f'{ROOT}/{mode}/{x}') for x in range(20,81,20) for mode in ['basic', 'fixed', 'communication', 'proportional']], ROOT)

    return (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan)#, (df_green_masa, df_sem_time_masa, df_trip_time_masa)

def isEquipped(attrs, sep):
    attrs_list = attrs.split(sep)
    for att in attrs_list:
        k,v = att.split('=')
        if k == 'drivelogic':
            return 'Bid' in v

def flagEquipped(df_green, df_sem_time, df_trip_time):
    df_trip_time["Equipped"] = df_trip_time["Attributes"].map(lambda x:isEquipped(x, ';'))

    df_trip_time = df_trip_time.astype({'% Equipped': 'int', 'Equipped': 'int', 'Person': 'int'})
    df_sem_time = df_sem_time.astype({'% Equipped': 'int', 'Agent': 'int'})
    df_green = df_green.astype({'% Equipped': 'int', 'Agent': 'int'})

    df_sem_time = df_sem_time.merge(df_trip_time, how='left', left_on=['run', 'Agent', '% Equipped', 'Mode'], right_on=['run', 'Person', '% Equipped', 'Mode'])[['Link', 'Time', 'Agent', 'Start', 'AgentAttrs', 'run', '% Equipped',
       'Mode', 'Equipped']]
    df_green = df_green.merge(df_trip_time, how='left', left_on=['run', 'Agent', '% Equipped', 'Mode'], right_on=['run', 'Person', '% Equipped', 'Mode'])[['Agent', 'Ratio', 'run', 'Equipped', 'Mode', '% Equipped']]
    return df_green, df_sem_time, df_trip_time

def equipped(manhattan, masa=None):
    # df_green_masa, df_sem_time_masa, df_trip_time_masa = flagEquipped(*masa)
    df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = flagEquipped(*manhattan)

    return (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan)#, (df_green_masa, df_sem_time_masa, df_trip_time_masa)

def traffic(t):
    # manhattan, masa = load(t)
    manhattan = load(t)
    # (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan), (df_green_masa, df_sem_time_masa, df_trip_time_masa) = equipped(manhattan, masa)
    (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan) = equipped(manhattan)
    if t == 'high':
        t = 'heavy'
    #draw_plots(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, df_green_masa, df_sem_time_masa, df_trip_time_masa, "% Equipped", f"Coexist - High {t}")
    draw_plots_box(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, "% Equipped", f"Coexist − {t} traffic")
    #draw_plots_box_detail(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, df_green_masa, df_sem_time_masa, df_trip_time_masa, "% Equipped", f"Coexist - High {t}")


In [ ]:
traffic('low')
traffic('medium')
traffic('high')

In [ ]:
def draw_plots_box(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, x, exp_str):
    df_trip_time_manhattan = df_trip_time_manhattan[df_trip_time_manhattan["Attributes"].str.contains('travelTime')]
    # df_trip_time_masa = df_trip_time_masa[df_trip_time_masa["Attributes"].str.contains('travelTime')]
    df_trip_time_manhattan['DTime'] = df_trip_time_manhattan["Attributes"].apply(lambda x:float(dict(map(lambda y:(y.split('=')[0], y.split('=')[1]) ,x.split(';')))["travelTime"]))
    # df_trip_time_masa['DTime'] = df_trip_time_masa["Attributes"].apply(lambda x:float(dict(map(lambda y:(y.split('=')[0], y.split('=')[1]) ,x.split(';')))["travelTime"]))   
    df_trip_time_manhattan["DDuration"] = df_trip_time_manhattan['Duration'] / df_trip_time_manhattan['DTime']
    # df_trip_time_masa["DDuration"] = df_trip_time_masa['Duration'] / df_trip_time_masa['DTime']

    # df_green_masa["Ratio100"] = df_green_masa["Ratio"] * 100
    df_green_manhattan["Ratio100"] = df_green_manhattan["Ratio"] * 100

    if exp_str == 'high traffic':
        exp_str = 'heavy traffic'

    fig = plt.figure(figsize=(20, 18), dpi = 400)
    #fig.suptitle(exp_str)
    axs = fig.subplots(3, 2)
    #axs = figs[0].subplots(2, 1)
    #figs[0].suptitle("Time To Cross")
    axs[0,0].set_xticklabels(axs[0,0].get_xticks())
    sns.boxplot(data=df_sem_time_manhattan, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,0], showfliers = False, linewidth=2)
    handles, labels = axs[0,0].get_legend_handles_labels()
    sns.stripplot(data=df_sem_time_manhattan, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,0], dodge=True, jitter=False, marker="d", zorder=0, size=3)
    axs[0,0].set(yscale = 'log')
    axs[0,0].legend().remove()
    axs[0,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[0,0].set_ylabel("Time (s)")
    axs[0,0].set_xlabel("% Connected")
    axs[0,0].set_title(f"Time to Cross − Manhattan − {exp_str} (a)")
    #ax = axs[0].subplot(1,2,2)
    #axs[1].xticks(rotation = 45)
    axs[0,1].set_xticklabels(axs[0,1].get_xticks())
    # sns.boxplot(data=df_sem_time_masa, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,1], showfliers = False, linewidth=2)
    handles, labels = axs[0,1].get_legend_handles_labels()
    # sns.stripplot(data=df_sem_time_masa, x=x, y='Time', hue='Mode', palette=palette, ax=axs[0,1], dodge=True, jitter=False, marker="d", zorder=0, size=3)
    axs[0,1].set(yscale = 'log')
    axs[0,1].legend().remove()
    axs[0,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #axs[1].figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.06), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[0,1].set_ylabel("Time (s)")
    axs[0,1].set_xlabel("% Connected")
    axs[0,1].set_title(f"Time to Cross − MASA − {exp_str} (d)")
    #axs[0].suptitle(exp_str + " - Time to cross traffic lights")
    #plt.figure(figsize=(8, 6), dpi = 1200)
    #plt.subplot(1,2,1)
    #plt.xticks(rotation = 45)
    #axs = figs[1].subplots(2, 1)
    #figs[1].suptitle("Green Traffic Lights Ratio")
    axs[1,0].set_xticklabels(axs[1,0].get_xticks())
    sns.boxplot(data=df_green_manhattan.query("Mode != 'fixed'"), x=x, y='Ratio100', hue='Mode', palette=palette, ax=axs[1,0], showfliers = False)
    handles, labels = axs[1,0].get_legend_handles_labels()
    axs[1,0].legend().remove()
    axs[1,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[1,0].set_ylabel("% of green traffic lights")
    axs[1,0].set_xlabel("% Connected")
    axs[1,0].set_title(f"Green Traffic Lights Percentage − Manhattan − {exp_str} (b)")
    #ax = plt.subplot(1,2,2)
    #plt.xticks(rotation = 45)
    axs[1,1].set_xticklabels(axs[1,1].get_xticks())
    # sns.boxplot(data=df_green_masa.query("Mode != 'fixed'"), x=x, y='Ratio100', hue='Mode', palette=palette, ax=axs[1,1], showfliers = False)
    #handles, labels = ax.get_legend_handles_labels()
    handles, labels = axs[1,1].get_legend_handles_labels()
    axs[1,1].legend().remove()
    axs[1,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #plt.figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.06), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[1,1].set_ylabel("% of green traffic lights")
    axs[1,1].set_xlabel("% Connected")
    axs[1,1].set_title(f"Green Traffic Lights Percentage − MASA − {exp_str} (e)")
    #plt.suptitle(exp_str + " - Encurred green traffic lights")
    #plt.figure(figsize=(8, 6), dpi = 1200)
    #plt.subplot(1,2,1)
    #plt.xticks(rotation = 45)
    #axs = figs[2].subplots(2, 1)
    #axs[0].set_xticklabels(axs[0].get_xticks(), rotation = 45)
    #figs[2].suptitle("Trip Time Slowdown")
    sns.boxplot(data=df_trip_time_manhattan, x=x, y='DDuration', hue='Mode', palette=palette, ax=axs[1,0], showfliers = False)
    axs[2,0].set(yscale = 'log')
    handles, labels = axs[2,0].get_legend_handles_labels()
    axs[2,0].legend().remove()
    axs[2,0].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[2,0].set_ylabel("Slowdown")
    axs[2,0].set_xlabel("% Connected")
    axs[2,0].set_title(f"Trip Time Slowdown − Manahattan − {exp_str} (c)")
    # qs = df_trip_time_manhattan.query("Mode == 'communication'")["DDuration"].quantile([0.25, 0.75])
    # iqr = qs[0.75] - qs[0.25]
    # axs[0].axis(ymin=qs[0.25] - 2*iqr, ymax=qs[0.75] + 1.5*iqr)
    #ax = plt.subplot(1,2,2)
    #plt.xticks(rotation = 45)
    axs[2,1].set_xticklabels(axs[2,1].get_xticks())
    # sns.boxplot(data=df_trip_time_masa, x=x, y='DDuration', hue='Mode', palette=palette, ax=axs[2,1], showfliers = False)
    axs[2,1].set(yscale = 'log')
    #handles, labels = ax.get_legend_handles_labels()
    handles, labels = axs[2,1].get_legend_handles_labels()
    axs[2,1].legend().remove()
    axs[2,1].legend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.2), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    #plt.figlegend(handles=handles, labels=map_labels(labels), loc = 'lower center', bbox_to_anchor=(0.5, -0.1), prop={'size': 9}, handlelength=3, ncol=4, labelspacing=0.0)
    axs[2,1].set_ylabel("Slowdown")
    axs[2,1].set_xlabel("% Connected")
    axs[2,1].set_title(f"Trip Time Slowdown − MASA − {exp_str} (f)")
    fig.tight_layout(h_pad=2.5)


In [ ]:
def load(system, suffix):
    # ROOT = f'output/manhattan/exp3traffic{traffic}'
    volumes = [1000, 2500, 4000]
    ROOT = f'output/manhattan/exp3{suffix}'
    modes = ['basic', 'fixed', 'communication', 'proportional'] if suffix == '' else ['communication', 'proportional']
    df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = get_dataframes([({'Traffic': traffic, 'Mode': mode}, f'{ROOT}/{mode if mode != "fixed" else "ftc"}/{traffic}traffic210000emergency_budget') for traffic in volumes for mode in modes], ROOT)
    df_green_manhattan['Mode'] = df_green_manhattan['Mode'].apply(lambda x: x if x == 'fixed' or x == 'basic' else x+system)
    df_sem_time_manhattan['Mode'] = df_sem_time_manhattan['Mode'].apply(lambda x: x if x == 'fixed' or x == 'basic' else x+system)
    df_trip_time_manhattan['Mode'] = df_trip_time_manhattan['Mode'].apply(lambda x: x if x == 'fixed' or x == 'basic' else x+system)
    df_green_manhattan['Traffic'] = df_green_manhattan['Traffic'].apply(lambda x: 'heavy' if x == 'high' else x)
    df_sem_time_manhattan['Traffic'] = df_sem_time_manhattan['Traffic'].apply(lambda x: 'heavy' if x == 'high' else x)
    df_trip_time_manhattan['Traffic'] = df_trip_time_manhattan['Traffic'].apply(lambda x: 'heavy' if x == 'high' else x)
    

    # ROOT = f'MASA/output/coexist_emergency{suffix}'
    # df_green_masa, df_sem_time_masa, df_trip_time_masa = get_dataframes([({'Traffic': x, 'Mode': mode}, f'{ROOT}/{mode}/{x}') for x in ['high', 'medium', 'low'] for mode in ['basic', 'fixed', 'communication', 'proportional']], ROOT)
    # df_green_masa['Mode'] = df_green_masa['Mode'].apply(lambda x: x if x == 'fixed' or x == 'basic' else x+system)
    # df_sem_time_masa['Mode'] = df_sem_time_masa['Mode'].apply(lambda x: x if x == 'fixed' or x == 'basic' else x+system)
    # df_trip_time_masa['Mode'] = df_trip_time_masa['Mode'].apply(lambda x: x if x == 'fixed' or x == 'basic' else x+system)
    # df_green_masa['Traffic'] = df_green_masa['Traffic'].apply(lambda x: 'heavy' if x == 'high' else x)
    # df_sem_time_masa['Traffic'] = df_sem_time_masa['Traffic'].apply(lambda x: 'heavy' if x == 'high' else x)
    # df_trip_time_masa['Traffic'] = df_trip_time_masa['Traffic'].apply(lambda x: 'heavy' if x == 'high' else x)

    return (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan)#, (df_green_masa, df_sem_time_masa, df_trip_time_masa)

def isEmergency(attrs, sep):
    attrs_list = attrs.split(sep)
    for att in attrs_list:
        k,v = att.split('=')
        if k == 'budget':
            return v == '210000'#'420000'

def flagEmergency(df_green, df_sem_time, df_trip_time):
    df_trip_time["Emergency"] = df_trip_time["Attributes"].map(lambda x:isEmergency(x, ';'))

    df_trip_time = df_trip_time.astype({'Person': 'int'})
    df_sem_time = df_sem_time.astype({'Agent': 'int'})
    df_green = df_green.astype({'Agent': 'int'})

    df_sem_time = df_sem_time.merge(df_trip_time, how='left', left_on=['run', 'Agent', 'Traffic', 'Mode'], right_on=['run', 'Person', 'Traffic', 'Mode'])[['Link', 'Time', 'Agent', 'Start', 'AgentAttrs', 'run', 'Traffic',
       'Mode', 'Emergency']]
    df_green = df_green.merge(df_trip_time, how='left', left_on=['run', 'Agent', 'Traffic', 'Mode'], right_on=['run', 'Person', 'Traffic', 'Mode'])[['Agent', 'Ratio', 'run', 'Emergency', 'Mode', 'Traffic']]
    return df_green, df_sem_time, df_trip_time

def emergency(manhattan, masa=None):
    # df_green_masa, df_sem_time_masa, df_trip_time_masa = flagEmergency(*masa)
    df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan = flagEmergency(*manhattan)

    return (df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan)#, (df_green_masa, df_sem_time_masa, df_trip_time_masa)

def t_emergency():
    manhattan_prop = load('', '')
    manhattan_prop = emergency(manhattan_prop)#, masa_prop)
    manhattan_no_prop = load(' NO Propagation', '_no_prop')
    manhattan_no_prop = emergency(manhattan_no_prop) #, masa_no_prop)
    df_green_manhattan = pd.concat([manhattan_prop[0], manhattan_no_prop[0]])
    df_sem_time_manhattan = pd.concat([manhattan_prop[1], manhattan_no_prop[1]])
    df_trip_time_manhattan = pd.concat([manhattan_prop[2], manhattan_no_prop[2]])
    # df_green_manhattan = manhattan_prop[0] # bogus
    # df_sem_time_manhattan = manhattan_prop[1] # bogus
    # df_trip_time_manhattan = manhattan_prop[2] # bogus
    # df_green_masa = pd.concat([masa_prop[0], masa_no_prop[0]])
    # df_sem_time_masa = pd.concat([masa_prop[1], masa_no_prop[1]])
    # df_trip_time_masa = pd.concat([masa_prop[2], masa_no_prop[2]])
    #draw_plots(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, df_green_masa, df_sem_time_masa, df_trip_time_masa, "% Equipped", f"Coexist - High {t}")
    # draw_plots_box(df_green_manhattan.query("Emergency == 1"), df_sem_time_manhattan.query("Emergency == 1"), df_trip_time_manhattan.query("Emergency == 1"), df_green_masa, df_sem_time_masa.query("Emergency == 1"), df_trip_time_masa.query("Emergency == 1"), "Traffic", f"Emergency")
    draw_plots_box(df_green_manhattan.query("Emergency == 1"), df_sem_time_manhattan.query("Emergency == 1"), df_trip_time_manhattan.query("Emergency == 1"), "Traffic", f"Emergency")
    #draw_plots_box_detail(df_green_manhattan, df_sem_time_manhattan, df_trip_time_manhattan, df_green_masa, df_sem_time_masa, df_trip_time_masa, "% Equipped", f"Coexist - High {t}")

In [ ]:

t_emergency()